# howmany

The following notebook gives a quick overview of how to use howmany.

In [1]:
pip install howmany

In [2]:
%load_ext jupyter_black

In [3]:
# import os
# import sys

# module_path = os.path.abspath(os.path.join("../src"))
# if module_path not in sys.path:
#     sys.path.append(module_path)

from IPython.display import display, HTML

display(HTML("<style>.container { width:99% !important; }</style>"))

In [4]:
import howmany
from howmany.utils import float_to_str

In [5]:
soccer_fields_in_germany_dict = howmany.compare(
    containers="Q183", entities="Q8524", pid="P2046", iso="en"
)

In [6]:
for k in soccer_fields_in_germany_dict.keys():
    amount = round(soccer_fields_in_germany_dict[k]["amount"], 2)
    print(
        f"You could fit {amount:,} {soccer_fields_in_germany_dict[k]['entity']}es inside {k}."
    )

You could fit 50,453,300.88 association football pitches inside Germany.


In [7]:
germanies_in_soccer_fields_dict = howmany.compare(
    containers="Q8524", entities="Q183", pid="P2046", iso="en"
)

In [8]:
for k in germanies_in_soccer_fields_dict.keys():
    amount = float_to_str(germanies_in_soccer_fields_dict[k]["amount"])
    print(
        f"You could fit {amount} {germanies_in_soccer_fields_dict[k]['entity']}s inside an {k}."
    )

You could fit 0.000000019820308731475912 Germanys inside an association football pitch.
